In [8]:
from scripts.models import *
from glob import glob
import pandas as pd
import os
from tqdm import tqdm

/usr/local/lib/python3.5/dist-packages/django/db/models/base.py:316: RuntimeWarning: Model 'server.video' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/usr/local/lib/python3.5/dist-packages/django/db/models/base.py:316: RuntimeWarning: Model 'server.pedestrian' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/usr/local/lib/python3.5/dist-packages/django/db/models/base.py:316: RuntimeWarning: Model 'server.pedpos' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/usr/local/lib/python3.5/dist-packages/django/db/models/base.py:316: Runt

In [9]:
DATA_DIR = '/app/data/vci-dataset-dut/data'
video_paths = glob('{}/videos/*.mp4'.format(DATA_DIR))

In [3]:
videos = [Video.infer_ffmpeg(p) for p in video_paths]

for v in videos:
    v.save()

In [10]:
videos = Video.objects.all()

In [12]:
for video in videos:
    base_name = '_'.join(os.path.basename(video.path).split('_')[:2])
    p2m = float(open('{}/ratios/{}_ratio_pixel2meter.txt'.format(DATA_DIR, base_name)).read())
    video.pixel2meter = p2m
    video.save()

In [54]:
for video in tqdm(videos[1:]):
    base_name = '_'.join(os.path.basename(video.path).split('_')[:2])

    veh_pos = []
    df = pd.read_csv('{}/trajectories_filtered/{}_traj_veh_filtered.csv'.format(DATA_DIR, base_name))
    veh = {}
    for _, row in df.iterrows():
        if not row.id in veh:
            veh[row.id] = Vehicle(video=video)
    Vehicle.objects.bulk_create(veh.values())

    for _, row in df.iterrows():
        veh_pos.append(VehPos(
            frame=row.frame,
            veh=veh[row.id],
            x=row.x_est,
            y=row.y_est,
            psi=row.psi_est,
            vel=row.vel_est
        ))
    VehPos.objects.bulk_create(veh_pos)  

    ped_pos = []
    df = pd.read_csv('{}/trajectories_filtered/{}_traj_ped_filtered.csv'.format(DATA_DIR, base_name))
    ped = {}
    for _, row in df.iterrows():
        if not row.id in ped:
            ped[row.id] = Pedestrian(video=video)
    Pedestrian.objects.bulk_create(ped.values())

    for _, row in df.iterrows():
        ped_pos.append(PedPos(
            frame=row.frame,
            ped=ped[row.id],
            x=row.x_est,
            y=row.y_est,
            vx=row.vy_est,
            vy=row.vx_est
        ))
    PedPos.objects.bulk_create(ped_pos)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [03:06<00:00,  9.63s/it]
